In [ ]:
!wget https://grammarly-nlp-data-public.s3.amazonaws.com/gector/xlnet_0_gector.th

--2021-01-29 16:05:31--  https://grammarly-nlp-data-public.s3.amazonaws.com/gector/xlnet_0_gector.th
Resolving grammarly-nlp-data-public.s3.amazonaws.com (grammarly-nlp-data-public.s3.amazonaws.com)... 52.216.249.44
Connecting to grammarly-nlp-data-public.s3.amazonaws.com (grammarly-nlp-data-public.s3.amazonaws.com)|52.216.249.44|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 482329108 (460M) [application/x-www-form-urlencoded]
Saving to: ‘xlnet_0_gector.th’

xlnet_0_gector.th   100%[===================>] 459.98M  44.3MB/s    in 11s     

2021-01-29 16:05:43 (41.9 MB/s) - ‘xlnet_0_gector.th’ saved [482329108/482329108]



In [ ]:
!git clone https://github.com/grammarly/gector

Cloning into 'gector'...
remote: Enumerating objects: 79, done.
remote: Total 79 (delta 0), reused 0 (delta 0), pack-reused 79
Unpacking objects: 100% (79/79), done.


In [ ]:
%%capture
!pip install -r /content/gector/requirements.txt

In [ ]:
!pip install -U transformers

  Using cached https://files.pythonhosted.org/packages/88/b1/41130a228dd656a1a31ba281598a968320283f48d42782845f6ba567f00b/transformers-4.2.2-py3-none-any.whl
  Found existing installation: transformers 2.2.2
    Uninstalling transformers-2.2.2:
      Successfully uninstalled transformers-2.2.2


In [ ]:
%cd /content/gector

/content/gector


In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

from predict import predict_for_file
from gector.gec_model import GecBERTModel
import importlib

import torch

In [ ]:
rtokenizer = AutoTokenizer.from_pretrained('roberta-large')
rmodel = AutoModelForMaskedLM.from_pretrained('roberta-large')

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-large and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model = GecBERTModel(vocab_path='data/output_vocabulary', model_paths=['/content/xlnet_0_gector.th'], model_name='xlnet', special_tokens_fix=0)

In [ ]:
def get_suggestions(text, no_suggestions=3):
  model_input = rtokenizer.encode(text, return_tensors='pt')
  mask_token_index = torch.where(model_input == rtokenizer.mask_token_id)[1]

  token_logits = rmodel(model_input)[0]
  mask_token_logits = token_logits[0, mask_token_index, :]

  top_tokens = torch.topk(mask_token_logits, no_suggestions, dim=1).indices[0].tolist()
  return [text.replace(rtokenizer.mask_token, rtokenizer.decode([token])) for token in top_tokens]

In [ ]:
def stringify(words):
  out = ""
  for word in words:
    out += word + " "
  return out

In [ ]:
def insert(words, index, ins=rtokenizer.mask_token):
  return words[:index] + [ins] + words[index:]

In [ ]:
def score_sentences(sentence):
  words = sentence.split()
  _, _, score = model.handle_batch([words])
  return score[0]

In [ ]:
def score_words(sentence):
  words = sentence.split()
  scores, _, _ = model.handle_batch([words])
  scores[0].pop(0)
  return scores[0]

In [ ]:
def get_bad_indices(sentence):
  scores = score_words(sentence)
  copy = scores.copy()
  scores.sort()
  lowest = scores[0: int(len(scores) * .20)]
  if len(lowest) < 3:
    lowest = scores[0:3]
  bad_indices = []
  for low in lowest:
    bad_indices.append(copy.index(low))
  
  return bad_indices

In [ ]:
def check_addition(sentence, bad_indices):
  corrections = []
  for index in bad_indices:
    words = sentence.split()
    words.pop(index)
    corrected = stringify(words)
    corrections.append((corrected, score_sentences(corrected)))
  return corrections

In [ ]:
def check_substitution(sentence, bad_indices, no_suggestions=1):
  corrections = []
  for index in bad_indices:
    words = sentence.split()
    words[index] = rtokenizer.mask_token
    string = stringify(words)
    for correction in get_suggestions(string, no_suggestions):
      corrections.append((correction, score_sentences(correction)))
  return corrections

In [ ]:
def check_deletion(sentence, bad_indices, no_suggestions=1):
  corrections = []
  for index in bad_indices:
    words = sentence.split()
    wordsl = insert(words, index)
    wordsr = insert(words, index+1)
    stringl = stringify(wordsl)
    stringr = stringify(wordsr)
    for correction in get_suggestions(stringr, no_suggestions):
      corrections.append((correction, score_sentences(correction)))
    for correction in get_suggestions(stringl, no_suggestions):
      corrections.append((correction, score_sentences(correction)))
  return corrections

In [ ]:
def driver(sentences):
  for sentence in sentences:
    bad_indices = get_bad_indices(sentence)
    print(f'Original sentence: {sentence}')
    print(f'Original score: {score_sentences(sentence)}')
    print('Addition:')
    print(check_addition(sentence, bad_indices))
    print('Substitution:')
    print(check_substitution(sentence, bad_indices))
    print('Deletion:')
    print(check_deletion(sentence, bad_indices))
    print('-' * 800)

In [ ]:
def best(corrections):
  best_score = 1
  best_correction = ''
  for correction, score in corrections:
    if score <= best_score:
      best_correction = correction
      best_score = score
  return best_correction, best_score

In [ ]:
def get_best(sentences):
  for sentence in sentences:
    bad_indices = get_bad_indices(sentence)
    out = []
    print(f'Original sentence: {sentence}, {score_sentences(sentence)}')
    out.append(check_addition(sentence, bad_indices))
    out.append(check_deletion(sentence, bad_indices))
    out.append(check_substitution(sentence, bad_indices))

  return best([best(out[0]), best(out[1]), best(out[2])])

In [ ]:
def iterate(sentence, iterations=5):
  curr = sentence
  for _ in range(iterations):
    curr = get_best([curr])[0]
  return curr

In [ ]:
get_best(["Because of diagonal elements are zeros , the components are null ."])

Original sentence: Because of diagonal elements are zeros , the components are null ., 0.856610119342804


('Because diagonal elements are zeros , the components are null . ',
 0.507723331451416)

In [ ]:
driver(["Because of diagonal elements are zeros , the components are null ."])

Original sentence: Because of diagonal elements are zeros , the components are null .
Original score: 0.856610119342804
Addition:
[('Because diagonal elements are zeros , the components are null . ', 0.507723331451416), ('Because of diagonal elements are , the components are null . ', 0.8308606743812561), ('Because of elements are zeros , the components are null . ', 0.8769802451133728)]
Substitution:
[('Because  the diagonal elements are zeros , the components are null . ', 0.5783601999282837), ('Because of diagonal elements are  null , the components are null . ', 0.8797251582145691), ('Because of  the elements are zeros , the components are null . ', 0.6306759715080261)]
Deletion:
[('Because of  the diagonal elements are zeros , the components are null . ', 0.6936379671096802), ('Because  all of diagonal elements are zeros , the components are null . ', 0.8822611570358276), ('Because of diagonal elements are zeros  and , the components are null . ', 0.8970226645469666), ('Because of

In [ ]:
get_best(["These are a read and write operation times ."])

Original sentence: These are a read and write operation times ., 0.9724825024604797


('These are a read and write operation  at times . ', 0.528466522693634)

In [ ]:
driver(["These are a read and write operation times ."])

Original sentence: These are a read and write operation times .
Original score: 0.9724825024604797
Addition:
[('These are a and write operation times . ', 0.9170173406600952), ('These are a read and operation times . ', 0.9514626860618591), ('These are a read and write times . ', 0.9260262846946716)]
Substitution:
[('These are a  read and write operation times . ', 0.9724825024604797), ('These are a read and  write operation times . ', 0.9724825024604797), ('These are a read and write  many times . ', 0.9441121220588684)]
Deletion:
[('These are a read  operation and write operation times . ', 0.9290616512298584), ('These are a  typical read and write operation times . ', 0.9499345421791077), ('These are a read and write  sequential operation times . ', 0.9189997911453247), ('These are a read and  a write operation times . ', 0.9715926051139832), ('These are a read and write operation  at times . ', 0.528466522693634), ('These are a read and write  sequential operation times . ', 0.9189

In [ ]:
get_best(["You have to make a decision based on 0 or 1 , oh okay ."])

Original sentence: You have to make a decision based on 0 or 1 , oh okay ., 0.4426235556602478


('You have to make a decision based on 0 or 1 , okay . ', 0.22449840605258942)

In [ ]:
driver(["You have to make a decision based on its belief , 0 or 1 oh okay ."])

Original sentence: You have to make a decision based on its belief , 0 or 1 oh okay .
Original score: 0.5556721091270447
Addition:
[('You have to make a decision based on its belief 0 or 1 oh okay . ', 0.8931667804718018), ('You have to make a decision based on its belief , 0 or 1 okay . ', 0.7104674577713013), ('You have to make a decision based on belief , 0 or 1 oh okay . ', 0.5169863700866699)]
Substitution:
[('You have to make a decision based on its belief . 0 or 1 oh okay . ', 0.39965569972991943), ('You have to make a decision based on its belief , 0 or 1  is okay . ', 0.721209704875946), ('You have to make a decision based on  your belief , 0 or 1 oh okay . ', 0.5393906831741333)]
Deletion:
[('You have to make a decision based on its belief ,  so 0 or 1 oh okay . ', 0.5589056015014648), ('You have to make a decision based on its belief  value , 0 or 1 oh okay . ', 0.6365249156951904), ('You have to make a decision based on its belief , 0 or 1 oh  well okay . ', 0.7708204984664

In [ ]:
get_best(["You're creating the image and then sticking it on these metadata feeds so it shows up good on amazon on shopify ."])

Original sentence: You're creating the image and then sticking it on these metadata feeds so it shows up good on amazon on shopify ., 0.7682225704193115


("You're creating the image and then sticking it on these metadata feeds so it shows up on amazon on shopify . ",
 0.4910700023174286)

In [ ]:
driver(["You're creating the image and then sticking it on these metadata feeds so it shows up good on amazon on shopify ."])

Original sentence: You're creating the image and then sticking it on these metadata feeds so it shows up good on amazon on shopify .
Original score: 0.7682225704193115
Addition:
[("You're creating the image and then sticking it on these metadata feeds so it shows up on amazon on shopify . ", 0.4910700023174286), ("You're creating the image and then sticking it on these metadata feeds so it shows up good on on shopify . ", 0.6988755464553833), ("You're creating the image and then sticking it on these metadata feeds so it shows up good on amazon on . ", 0.888641893863678), ("You're the image and then sticking it on these metadata feeds so it shows up good on amazon on shopify . ", 0.8674836158752441)]
Substitution:
[("You're creating the image and then sticking it on these metadata feeds so it shows up  both on amazon on shopify . ", 0.9275521039962769), ("You're creating the image and then sticking it on these metadata feeds so it shows up good on  display on shopify . ", 0.767207026481

In [ ]:
get_best(["You're creating the image and then sticking it on these metadata feeds so it shows up good on amazon and shopify ."])

Original sentence: You're creating the image and then sticking it on these metadata feeds so it shows up good on amazon and shopify ., 0.8271904587745667


("You're creating the image and then sticking it on these metadata feeds so it shows up good on  google and shopify . ",
 0.7844880223274231)

In [ ]:
get_best(["A perfectly good mine sentence is not hard to come up with ."])

Original sentence: A perfectly good mine sentence is not hard to come up with ., 0.9786275029182434


('A perfectly good sentence is not hard to come up with . ',
 0.0014121211133897305)

In [ ]:
driver(["A perfectly good word sentence is not hard to come up with ."])

Original sentence: A perfectly good word sentence is not hard to come up with .
Original score: 0.8227465748786926
Addition:
[('A perfectly good sentence is not hard to come up with . ', 0.0014121211133897305), ('A perfectly word sentence is not hard to come up with . ', 0.9813939332962036), ('A perfectly good word is not hard to come up with . ', 0.004702916368842125)]
Substitution:
[('A perfectly good  English sentence is not hard to come up with . ', 0.003849945031106472), ('A perfectly  three word sentence is not hard to come up with . ', 0.7256753444671631), ('A perfectly good word  that is not hard to come up with . ', 0.021979404613375664)]
Deletion:
[('A perfectly good word  or sentence is not hard to come up with . ', 0.001316220499575138), ('A perfectly good  one word sentence is not hard to come up with . ', 0.6849609613418579), ('A perfectly good  one word sentence is not hard to come up with . ', 0.6849609613418579), ('A perfectly  constructed good word sentence is not har

In [ ]:
model.handle_batch(["A perfectly good word sentence is not hard to come up with .".split()])

([[0.9993915557861328,
   0.9976759552955627,
   0.9925858378410339,
   0.9478182792663574,
   0.3237801194190979,
   0.9791347980499268,
   0.9995473027229309,
   0.9987718462944031,
   0.9995954632759094,
   0.9999909400939941,
   0.9997747540473938,
   0.999744713306427,
   0.9992377758026123,
   0.998079776763916]],
 [[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [0.8227465748786926])

In [ ]:
get_best(["You cannot count the number one of stars in the sky ."])

Original sentence: You cannot count the number one of stars in the sky ., 0.9175732731819153


('You cannot count the number   of stars in the sky . ', 0.008997755125164986)

In [ ]:
model.handle_batch(["You cannot count the number one of stars in the sky .".split()])

([[0.9997448325157166,
   0.9998155236244202,
   0.9947867393493652,
   0.9994344115257263,
   0.9266901612281799,
   0.9646723866462708,
   0.8570123910903931,
   0.886479377746582,
   0.9958611130714417,
   0.9999306201934814,
   0.999940037727356,
   0.9997171759605408,
   0.99698406457901]],
 [[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]],
 [0.9175732731819153])

In [ ]:
get_best(["You cannot count the number of stars stuck in the sky ."])

Original sentence: You cannot count the number of stars stuck in the sky ., 0.06510264426469803


('You cannot count the number of stars in the sky . ', 0.008997755125164986)

In [ ]:
get_best(["You can't count the all number of stars in the sky ."])

Original sentence: You can't count the all number of stars in the sky ., 0.9386579990386963


("You can't count the number of stars in the sky . ", 0.01278283167630434)

In [ ]:
get_best(["Des and Troy are the names fire of my pets ."])

Original sentence: Des and Troy are the names fire of my pets ., 0.9639774560928345


('Des and Troy are the names of my pets . ', 0.003129757009446621)

In [ ]:
driver(["Des and Troy are the names dog of my pets ."])

Original sentence: Des and Troy are the names dog of my pets .
Original score: 0.9755066633224487
Addition:
[('Des and Troy are the names dog my pets . ', 0.9293693900108337), ('Des and Troy are the dog of my pets . ', 0.8957498073577881), ('Des and Troy are the names dog of my . ', 0.9664055705070496)]
Substitution:
[('Des and Troy are the names dog  of my pets . ', 0.9755066633224487), ('Des and Troy are the  best dog of my pets . ', 0.8994035124778748), ('Des and Troy are the names dog of my  family . ', 0.9834161996841431)]
Deletion:
[('Des and Troy are the names dog of  all my pets . ', 0.9775081276893616), ('Des and Troy are the names dog  two of my pets . ', 0.9845609664916992), ('Des and Troy are the names  of dog of my pets . ', 0.9397597908973694), ('Des and Troy are the  real names dog of my pets . ', 0.9615486264228821), ('Des and Troy are the names dog of my pets . . ', 0.9766221642494202), ('Des and Troy are the names dog of my  two pets . ', 0.976362407207489)]
---------

In [ ]:
model.handle_batch(["Des and Troy are the names dog of my pets .".split()])

([[0.9797545075416565,
   0.9984961748123169,
   0.999744713306427,
   0.9984315037727356,
   0.9929258823394775,
   0.9253250956535339,
   0.8665928840637207,
   0.9173587560653687,
   0.6407230496406555,
   0.9556306600570679,
   0.9059197902679443,
   0.9978405237197876]],
 [[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]],
 [0.9755066633224487])

In [ ]:
get_best(["I buy shoes and sneakers every other damn day !"])

Original sentence: I buy shoes and sneakers every other damn day !, 0.11492287367582321


('I buy shoes and sneakers every other day ! ', 0.008237226866185665)

In [ ]:
get_best(["I buy shoes him and sneakers every other day !"])

Original sentence: I buy shoes him and sneakers every other day !, 0.9079687595367432


('I buy shoes and sneakers every other day ! ', 0.008237226866185665)

In [ ]:
driver(["I buy shoes and sneakers feet every other day !"])

Original sentence: I buy shoes and sneakers feet every other day !
Original score: 0.844481348991394
Addition:
[('I buy shoes and sneakers every other day ! ', 0.008237226866185665), ('I buy shoes and feet every other day ! ', 0.13250450789928436), ('I buy shoes sneakers feet every other day ! ', 0.9192432165145874)]
Substitution:
[('I buy shoes and sneakers  almost every other day ! ', 0.006117815617471933), ('I buy shoes and  wash feet every other day ! ', 0.6390203237533569), ('I buy shoes  for sneakers feet every other day ! ', 0.8282755017280579)]
Deletion:
[('I buy shoes and sneakers feet  size every other day ! ', 0.7187116146087646), ('I buy shoes and sneakers  for feet every other day ! ', 0.7279530763626099), ('I buy shoes and sneakers  for feet every other day ! ', 0.7279530763626099), ('I buy shoes and  wear sneakers feet every other day ! ', 0.8143393993377686), ('I buy shoes and  wear sneakers feet every other day ! ', 0.8143393993377686), ('I buy shoes  feet and sneakers

In [ ]:
model.handle_batch(["I buy shoes and sneakers feet every other day !".split()])

([[0.9991279244422913,
   0.9982174038887024,
   0.9773649573326111,
   0.9941845536231995,
   0.9742037057876587,
   0.9042534232139587,
   0.7993203997612,
   0.9958051443099976,
   0.9946523904800415,
   0.9984087347984314,
   0.9959859251976013]],
 [[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]],
 [0.844481348991394])

In [ ]:
model.handle_batch(["I buy shoes and sneakers every other damn day !".split()])

([[0.9996254444122314,
   0.9991593360900879,
   0.9982988238334656,
   0.9996269941329956,
   0.9999001026153564,
   0.997752845287323,
   0.9965100884437561,
   0.987461268901825,
   0.9261882305145264,
   0.9989066123962402,
   0.9976708292961121]],
 [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
 [0.11492287367582321])